In [1]:
import pandas as pd
import warnings
import numpy as np
import optuna
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.mixture import GaussianMixture

warnings.filterwarnings("ignore")

In [2]:
train_data = pd.read_csv('train.csv', header=None)


train_data.head()
y = pd.read_csv('train-target.csv', header=None)
y.rename({0: "target"}, axis = 'columns', inplace=True)
train = pd.concat([train_data, y], axis = 1)
train.columns = train.columns.astype(str)


In [3]:
test = pd.read_csv('test.csv', header=None)
test.columns = test.columns.astype(str)
train.columns = train.columns.astype(str)


In [4]:
train['target'] = train['target'].astype(int)
train_new = train.drop(['9', '15', '16', '17', '22', '26'], axis = 1)
test_new = test.drop(['9', '15', '16', '17', '22', '26'], axis = 1)


In [5]:
columns = test_new.columns
test_log = test_new.copy()
for column1 in columns:
    for column2 in columns:
        if column1 != column2:
            test_log[f'{column1}' + '/' + f'{column2}'] = test[column1] / test[column2]
            test_log[f'{column2}' + '/' + f'{column1}'] = test[column2] / test[column1]
            test_log[f'{column1}' + '*' + f'{column2}'] = test[column1] * test[column2]


            
columns = train_new.drop(['target'], axis = 1).columns
train_log = train_new.copy()
for column1 in columns:
    for column2 in columns:
        if column1 != column2:
            train_log[f'{column1}' + '/' + f'{column2}'] = train[column1] / train[column2]
            train_log[f'{column2}' + '/' + f'{column1}'] = train[column2] / train[column1]
            train_log[f'{column1}' + '*' + f'{column2}'] = train[column1] * train[column2]

In [18]:
test_log.shape

(2000, 1133)

In [19]:
train_log['max'] = train_new.drop(['target'], axis = 1).max(axis = 1)
test_log['max'] = test_new.max(axis = 1)

train_log['mean'] = train_new.drop(['target'], axis = 1).mean(axis = 1)
test_log['mean'] = test_new.mean(axis = 1)


train_log['min'] = train_new.drop(['target'], axis = 1).min(axis = 1)
test_log['min'] = test_new.min(axis = 1)


train_log['std'] = train_new.drop(['target'], axis = 1).std(axis = 1)
test_log['std'] = test_new.std(axis = 1)

# Reserch

In [30]:
my_favorite_cols = ['10', '29', '13', '28', '14', '7']

In [31]:
plot_cols = np.r_[my_favorite_cols]
plot_data_tr = X_tr_[plot_cols].copy()
plot_data_val = X_val_[plot_cols].copy()
plot_data_tst = test_new[plot_cols].copy()

# СКЕЙЛИНГ
means = plot_data_tr[my_favorite_cols].mean(axis=0)
stds = plot_data_tr[my_favorite_cols].std(axis=0)

plot_data_tr.loc[:, my_favorite_cols] = (plot_data_tr.loc[:, my_favorite_cols] - means) / stds
plot_data_tst.loc[:, my_favorite_cols] = (plot_data_tst.loc[:, my_favorite_cols] - means) / stds
plot_data_val.loc[:, my_favorite_cols] = (plot_data_val.loc[:, my_favorite_cols] - means) / stds

plot_data_scaled = pd.concat([plot_data_tr, plot_data_tst], axis=0, sort=False, ignore_index=True)

In [15]:
from reductors import *
from utils import plot_dim_reduction
from collections import namedtuple


hue_info = namedtuple('hue_info', ['field_name', 'is_categorical'])


def plot_standard_embeddings(plot_data, features, return_results=False):
    '''
    рисует "какие-то" эмбеддинги с hue='smoking' на одних и тех же фичах features
    '''
    mapper_dict = {
        'tsne 2D perplexity=60 exaggeration=4 dof=0.5': {
            'params': {
                'n_jobs': -1,
                'perplexity': 60,
                'verbose': False,
                'n_components': 2,
                'exaggeration': 3,
                'dof': 0.1,
                'neighbors': 'pynndescent',
                'metric': 'cosine',

            },
            'func': make_tsne,
        },
    }

    res = plot_dim_reduction(
        data=plot_data,
        mapper_dict=mapper_dict,
        default_features=features,
        default_hue_info=None,
        row_height=450,
        return_results=return_results
    )
    return res

results = plot_standard_embeddings(plot_data_tr, my_favorite_cols, return_results=True)

Loading BokehJS ...

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [05:23<00:00, 323.00s/it]


In [16]:
tsne_mapper_embedding = results['tsne 2D perplexity=60 exaggeration=4 dof=0.5']['embedding']

tsne_tst_embedding = tsne_mapper_embedding.transform(plot_data_tst, perplexity=60)
tsne_val_embedding = tsne_mapper_embedding.transform(plot_data_val, perplexity=60)

In [17]:
import hdbscan as hd# немного долго импортируется в первый раз

hdbscan_ = hd.HDBSCAN(min_samples=10, min_cluster_size=30, prediction_data = True)
train_clusters = hdbscan_.fit_predict(results['tsne 2D perplexity=60 exaggeration=4 dof=0.5']['embedding'])
clusterer = hdbscan_.fit(results['tsne 2D perplexity=60 exaggeration=4 dof=0.5']['embedding'])

In [18]:
val_clusters, clusters = hd.approximate_predict(clusterer, tsne_val_embedding)
tst_clusters, clusters = hd.approximate_predict(clusterer, tsne_tst_embedding)


In [19]:
X_tr_['clusters'] = train_clusters
X_val_['clusters'] = val_clusters
test_new['clusters'] = tst_clusters

# Model

In [20]:
from sklearn.model_selection import train_test_split

X_tr_, X_val_, y_tr_, y_val_ = train_test_split(train_new.drop(['target'], axis = 1), train_new.target, test_size = 0.2, random_state=42)
X_tr_log_, X_val_log_, _, _ = train_test_split(train_log.drop(['target'], axis = 1), train_log.target, test_size = 0.2, random_state=42)

In [10]:
np.random.seed(42)

In [11]:
params_ = {'objective': 'binary', 
              'metric': 'binary_logloss', 
              'boosting_type': 'dart',
              'eta': 0.5, 
              'max_depth': 10,
              'verbosity': -1, 
              'num_boost_round':400,
              'random_state': 42,
}

In [28]:
params = {'alpha_lgb': 0.5989736184067831,
 'alpha_svc': 0.9974772778514516,
 'alpha_log': 0.4583533197682535,
 'alpha_knn': 0.7037588003920892,
 'alpha_gauss': 0.0028275875889342016,
 'alpha_reg': 0.27064683997450345,
 'alpha_knn_2': 0.3,
 'alpha_ber': 0.014678200472675844}

In [33]:
alpha_lgb = params['alpha_lgb']
alpha_svc = params['alpha_svc']
alpha_log = params['alpha_log']
alpha_knn = params['alpha_knn']
alpha_gauss = params['alpha_gauss']
alpha_reg = params['alpha_reg']
alpha_knn_2 = params['alpha_knn_2']
alpha_ber = params['alpha_ber']

In [34]:
dataset = lgb.Dataset(X_tr_log_, y_tr_)
lgb_model = lgb.train(params_, dataset)
y_lgb = lgb_model.predict(X_val_log_)


SVC_model = SVC(C = 0.7, kernel='rbf', random_state = 42, probability = True).fit(X_tr_, y_tr_)
y_SVC = SVC_model.predict_proba(X_val_)[:, 1]


Log_model = LogisticRegression(penalty = 'l1', random_state = 42, solver = 'liblinear').fit(X_tr_log_, y_tr_)
y_log = Log_model.predict_proba(X_val_log_)[:, 1]


knn_model = KNeighborsClassifier(n_neighbors=60, metric = 'cosine',  weights = 'distance').fit(X_tr_, y_tr_)
y_knn = knn_model.predict_proba(X_val_)[:, 1]


knn_model_euc = KNeighborsClassifier(n_neighbors=35, metric = 'euclidean', weights = 'distance').fit(X_tr_, y_tr_)
y_knn_euc = knn_model.predict_proba(X_val_)[:, 1]


gauss_model = GaussianNB().fit(X_tr_, y_tr_)
y_gauss = gauss_model.predict_proba(X_val_)[:, 1]


linear_model = LinearRegression().fit(X_tr_log_, y_tr_)
y_reg = linear_model.predict(X_val_log_)


bernulli_model = BernoulliNB().fit(X_tr_log_, y_tr_)
y_ber = bernulli_model.predict_proba(X_val_log_)[:, 1]



y_final = y_knn_euc * alpha_knn_2 + alpha_lgb * y_lgb + alpha_svc * y_SVC + alpha_log * y_log + alpha_knn * y_knn + alpha_gauss * y_gauss + y_reg  * alpha_reg + alpha_ber * y_ber


roc_auc_score(y_val_, y_final)

0.8343690334666234

In [35]:
dataset = lgb.Dataset(X_tr_log_, y_tr_)
lgb_model = lgb.train(params_, dataset)
y_lgb = lgb_model.predict(test_log)


SVC_model = SVC(C = 0.7, kernel='rbf', random_state = 42, probability = True).fit(X_tr_, y_tr_)
y_SVC = SVC_model.predict_proba(test_new)[:, 1]


Log_model = LogisticRegression(penalty = 'l1', random_state = 42, solver = 'liblinear').fit(X_tr_log_, y_tr_)
y_log = Log_model.predict_proba(test_log)[:, 1]


knn_model = KNeighborsClassifier(n_neighbors=60, metric = 'cosine',  weights = 'distance').fit(X_tr_, y_tr_)
y_knn = knn_model.predict_proba(test_new)[:, 1]


knn_model_euc = KNeighborsClassifier(n_neighbors=35, metric = 'euclidean', weights = 'distance').fit(X_tr_, y_tr_)
y_knn_euc = knn_model.predict_proba(test_new)[:, 1]


gauss_model = GaussianNB().fit(X_tr_, y_tr_)
y_gauss = gauss_model.predict_proba(test_new)[:, 1]


linear_model = LinearRegression().fit(X_tr_log_, y_tr_)
y_reg = linear_model.predict(test_log)


bernulli_model = BernoulliNB().fit(X_tr_log_, y_tr_)
y_ber = bernulli_model.predict_proba(test_log)[:, 1]



y_final = y_knn_euc * alpha_knn_2 + alpha_lgb * y_lgb + alpha_svc * y_SVC + alpha_log * y_log + alpha_knn * y_knn + alpha_gauss * y_gauss + y_reg  * alpha_reg + alpha_ber * y_ber


roc_auc_score(y_val_, y_final)

0.5006414336091198

In [38]:
df_subm = pd.DataFrame({'y': y_final})

In [39]:
name = 'ensembley_0.8345_robust'
df_subm.to_csv(f'{name}.csv', index=False, header=False)